In [1]:
#Monitoramento Vitor Luiz
import pandas as pd
from dash import Dash, html, dcc, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash_daq as daq
from dash.dependencies import Input, Output, State
import plotly.io as pio
import dash
import dash_bootstrap_components as dbc
from datetime import datetime, timezone
from pytz import timezone
from plotly.subplots import make_subplots
import oficial_func as of

In [ ]:
# HEX COLORS
# https://htmlcolorcodes.com
teste = "#9C1FBD"
teste1 = "#f70d1a"
teste2 = "#EEEE0A"
teste3 = "#3DEC0A"

In [2]:
df_UFF = of.importa_df("https://thingspeak.com/channels/1670099/")

df_UFF


created_at    object
entry_id       int64
field1         int64
field2         int64
field3         int64
field4         int64
field5         int64
field6         int64
field7         int64
dtype: object
<bound method NDFrame.describe of                  created_at  entry_id  field1  field2  field3  field4  field5  \
0   2023-04-03 18:26:22 UTC     26617      -1      -1      -1    3653     167   
1   2023-04-03 18:36:19 UTC     26618      -1      -1      -1    3653     165   
2   2023-04-03 18:56:17 UTC     26619      -1      -1      -1    3653     165   
3   2023-04-03 19:26:14 UTC     26620      -1      -1      -1    3653     168   
4   2023-04-03 19:36:13 UTC     26621      -1      -1      -1    3653     167   
..                      ...       ...     ...     ...     ...     ...     ...   
95  2023-04-05 23:21:06 UTC     26712      -1      -1      -1    3653     165   
96  2023-04-05 23:31:05 UTC     26713      -1      -1      -1    3653     171   
97  2023-04-05 23:41:05 UTC     26

,Data,entry_id,Ax,Ay,Az,Temperatura,Umidade1,Umidade2,Umidade3
Data,,,,,,,,,
2023-04-03 15:26:22-03:00,2023-04-03 18:26:22+00:00,26617,-1,-1,-1,3653,167,355,77
2023-04-03 15:36:19-03:00,2023-04-03 18:36:19+00:00,26618,-1,-1,-1,3653,165,355,75
2023-04-03 15:56:17-03:00,2023-04-03 18:56:17+00:00,26619,-1,-1,-1,3653,165,354,74
2023-04-03 16:26:14-03:00,2023-04-03 19:26:14+00:00,26620,-1,-1,-1,3653,168,356,77
2023-04-03 16:36:13-03:00,2023-04-03 19:36:13+00:00,26621,-1,-1,-1,3653,167,357,77
...,...,...,...,...,...,...,...,...,...
2023-04-05 20:21:06-03:00,2023-04-05 23:21:06+00:00,26712,-1,-1,-1,3653,165,425,83
2023-04-05 20:31:05-03:00,2023-04-05 23:31:05+00:00,26713,-1,-1,-1,3653,171,422,87
2023-04-05 20:41:05-03:00,2023-04-05 23:41:05+00:00,26714,-1,-1,-1,3653,175,423,85


In [3]:
teste = of.alarme_ACeT(df_UFF)
print(teste)

6


In [ ]:
#Candidatos a parâmetros da função
n1_u3 = 30
n2_u3 = 10
diferença_u3 = -10
col_u3 = "Umidade3"

In [ ]:
ultima_info = df_UFF.tail(1)
print(ultima_info)
ultima_info.describe
ultima_info

In [ ]:
ultima_info[col_u3]
ultima_info.loc[ultima_info[col_u3] < 50, "cor"] = "#9C1FBD" #roxo
ultima_info.loc[((ultima_info[col_u3] >= 50) & (ultima_info[col_u3] <= 100)), "cor"] = "#f70d1a" #vermelho
ultima_info.loc[((ultima_info[col_u3] > 100) & (ultima_info[col_u3] <= 200)), "cor"] = "#EEEE0A" #amarelo
ultima_info.loc[ultima_info[col_u3] > 200, "cor"] = "#3DEC0A" #verde
print(ultima_info[col_u3])
ultima_info["cor"]

In [ ]:
valor = ultima_info.iloc[0]["cor"]
valor

In [ ]:
media_n1 = df_UFF[col_u3].tail(n1_u3).mean()
media_n2 = df_UFF[col_u3].tail(n2_u3).mean()

print(media_n1)
print(media_n2)

media_n2 - media_n1

In [ ]:
#Se a subtração das médias derem negativo significa que o valor de U3 está diminuindo e logo está piorando a situação
if ((media_n2 - media_n1) <= diferença_u3):
    if valor == "#3DEC0A": #Quando verde ele vai para amarelo
        valor = "#EEEE0A"
    elif valor == "#EEEE0A": #Quando amarelo vai para vermelho
        valor = "#f70d1a"
    elif valor == "#f70d1a": #Quando vermelho vai para roxo
        valor = "#9C1FBD"
    else:
        valor = "#9C1FBD"

valor